# Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Dropout, Flatten, Dense
import torch
import torchvision
from PIL import Image
from tensorflow.keras.models import load_model

from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
RootDir = '/content/drive/MyDrive/STAT288_Project/'

# Data

In [ ]:
# Create augmented data with different rotations
def augment_data_rotation(x, y):
    augmented_x = []
    augmented_y = []
    for i in range(len(x)):
        curr = x[i]
        for _ in range(4):
            # flips
            flip_x = np.flip(curr, axis=0)
            flip_y = np.flip(curr, axis=1)
            flip_xy = np.flip(flip_x, axis=1)
            augmented_x.append(flip_x)
            augmented_y.append(y[i])
            augmented_x.append(flip_y)
            augmented_y.append(y[i])
            augmented_x.append(flip_xy)
            augmented_y.append(y[i])

            # rotations
            augmented_x.append(curr)
            augmented_y.append(y[i])
            curr = np.rot90(curr)
    return (augmented_x, augmented_y)

In [ ]:
# Create augmented data with different contrast and hue
def augment_data_contrast(x, y):
    augmented_x = []
    augmented_y = []
    for i in range(len(x)):
        im = Image.fromarray(np.uint8(x[i]))
        augmented_x.append(x[i])
        augmented_y.append(y[i])
        new_img = torchvision.transforms.ColorJitter(brightness=0, contrast=0.5, saturation=0, hue=0.5)(im)
        new_img = np.array(new_img)
        augmented_x.append(new_img)
        augmented_y.append(y[i])
    return (augmented_x, augmented_y)

In [ ]:
# Load initial data
x_train = np.load(RootDir + 'SpaceNet/sat_train.npy').astype('float32')
init_y_train = np.load(RootDir + 'SpaceNet/bul_train.npy').astype('float32')
x_test = np.load(RootDir + 'SpaceNet/sat_test.npy').astype('float32')
init_y_test = np.load(RootDir + 'SpaceNet/bul_test.npy').astype('float32')

print("x_train shape", x_train.shape)
print("init_y_train shape", init_y_train.shape)
print("x_test shape", x_test.shape)
print("init_y_test shape", init_y_test.shape)

y_train = np.sum(init_y_train, axis=(1, 2, 3))
y_test = np.sum(init_y_test, axis=(1, 2, 3))
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

In [ ]:
# Create augmented data for rotations
augmented_rotation_x_train, augmented_rotation_y_train = augment_data_rotation(x_train, y_train)
augmented_rotation_x_test, augmented_rotation_y_test = augment_data_rotation(x_test, y_test)

augmented_rotation_x_train = np.asarray(augmented_rotation_x_train)
augmented_rotation_y_train = np.asarray(augmented_rotation_y_train)
augmented_rotation_x_test = np.asarray(augmented_rotation_x_test)
augmented_rotation_y_test = np.asarray(augmented_rotation_y_test)

print("augmented_x_train shape", augmented_rotation_x_train.shape)
print("augmented_y_train shape", augmented_rotation_y_train.shape)

In [ ]:
augmented_contrast_x_train, augmented_contrast_y_train = augment_data_contrast(x_train, y_train)
augmented_contrast_x_test, augmented_contrast_y_test = augment_data_contrast(x_test, y_test)

augmented_contrast_x_train = np.asarray(augmented_contrast_x_train)
augmented_contrast_y_train = np.asarray(augmented_contrast_y_train)
augmented_contrast_x_test = np.asarray(augmented_contrast_x_test)
augmented_contrast_y_test = np.asarray(augmented_contrast_y_test)

print("augmented_x_train shape", augmented_contrast_x_train.shape)
print("augmented_y_train shape", augmented_contrast_y_train.shape)

# Display Data


In [ ]:
idx = 0
plt.imshow(x_test[idx, :, :, :].astype('uint8'))
plt.show()
plt.imshow(init_y_test[idx, :, :, 0].astype('uint8'))
plt.show()
print(y_test[idx])

In [ ]:
idx = 1
plt.imshow(augmented_contrast_x_test[idx, :, :, :].astype('uint8'))
plt.show()

# U-Net

In [ ]:
x_in = Input(shape=(128, 128, 3))

x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_in)
x_temp = Dropout(0.25)(x_temp)
x_skip1 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip1)
x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip2 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip2)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip3 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip3)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Flatten()(x_temp)
x_out = Dense(units=1, activation=None, use_bias=False)(x_temp)

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
model.save_weights("/content/drive/My Drive/STAT288_Project/DNN_model_original_weights.keras")

# Hyperparameters

In [ ]:
NumEpochs = 75
BatchSize = 32

# Train Original

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=NumEpochs, batch_size=BatchSize, verbose=1)

In [ ]:
# Save the model
model.save("/content/drive/My Drive/STAT288_Project/DNN_model_original.keras")

In [ ]:
# Load the model
model = load_model("/content/drive/My Drive/STAT288_Project/DNN_model_original.keras")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Original Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Inference Original

In [ ]:
# Prediction over the test dataset
pred_test = model.predict(augmented_rotation_x_test)
stddev = []
for i in range(len(x_test)):
    out = [pred_test[16 * i + 4 * 0 + 3], pred_test[16 * i + 4 * 1 + 3], pred_test[16 * i + 4 * 2 + 3], pred_test[16 * i + 4 * 3 + 3]]
    stddev.append(np.std(out))
print(np.average(stddev))

# Train Rotation

In [ ]:
model.load_weights("/content/drive/My Drive/STAT288_Project/DNN_model_original_weights.keras")

In [ ]:
augmented_history = model.fit(augmented_rotation_x_train, augmented_rotation_y_train, validation_data=(x_test, y_test), epochs=NumEpochs, batch_size=BatchSize, verbose=1)

In [ ]:
# Save the model
model.save("/content/drive/My Drive/STAT288_Project/DNN_model_augmented_rotation.keras")

In [ ]:
# Load the model
model = load_model("/content/drive/My Drive/STAT288_Project/DNN_model_augmented_rotation.keras")

In [ ]:
plt.plot(augmented_history.history['loss'])
plt.plot(augmented_history.history['val_loss'])
plt.title('Augmented Rotation Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Inference Rotation

In [ ]:
# Prediction over the test dataset
pred_test = model.predict(augmented_rotation_x_test)
stddev = []
for i in range(len(x_test)):
    out = [pred_test[16 * i + 4 * 0 + 3], pred_test[16 * i + 4 * 1 + 3], pred_test[16 * i + 4 * 2 + 3], pred_test[16 * i + 4 * 3 + 3]]
    stddev.append(np.std(out))
print(np.average(stddev))

# Train Contrast

In [ ]:
model.load_weights("/content/drive/My Drive/STAT288_Project/DNN_model_original_weights.keras")

In [ ]:
augmented_contrast_history = model.fit(augmented_contrast_x_train, augmented_contrast_y_train, validation_data=(x_test, y_test), epochs=NumEpochs, batch_size=BatchSize, verbose=1)

In [ ]:
# Save the model
model.save("/content/drive/My Drive/STAT288_Project/DNN_model_augmented_contrast.keras")

In [ ]:
# Load the model
model = load_model("/content/drive/My Drive/STAT288_Project/DNN_model_augmented_contrast.keras")

In [ ]:
plt.plot(augmented_contrast_history.history['loss'])
plt.plot(augmented_contrast_history.history['val_loss'])
plt.title('Augmented Contrast Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()